In [1]:
%load_ext autoreload
%autoreload 2
import gc
import re
import pandas as pd
from __future__ import print_function, division
import numpy as np
pd.set_option('max_columns', 200)
pd.set_option('max_rows', 200)
import os
import sys
HOME = os.path.expanduser("~")
sys.path.append(f'{HOME}/kaggle/data_analysis/library')
import utils
from utils import get_categorical_features, get_numeric_features, reduce_mem_usage, elo_save_feature
from preprocessing import get_dummies
import datetime

from tqdm import tqdm
import time
import sys
from joblib import Parallel, delayed

os.listdir('../input/')
key = 'card_id'
target = 'target'
ignore_list = [key, target, 'merchant_id', 'first_avtive_month']

In [3]:
train_test = utils.read_df_pkl('../input/base_term*0*')
df_hist = utils.read_df_pkl('../input/histori*0*')
df_new = utils.read_df_pkl('../input/new_mer*0*')

train_test = utils.reduce_mem_usage(train_test)
df_hist  = utils.reduce_mem_usage(df_hist )
df_new   = utils.reduce_mem_usage(df_new  )

Mem. usage decreased to 30.42 Mb (27.9% reduction)
Mem. usage decreased to 1971.22 Mb (0.0% reduction)
Mem. usage decreased to 129.17 Mb (0.0% reduction)


In [4]:
# df_hist = df_hist.set_index(key).join(train_test.set_index(key)).reset_index()
# df_new = df_new.set_index(key).join(train_test.set_index(key)).reset_index()
for df in [df_hist, df_new]:
    df['authorized_flag'] = df['authorized_flag'].map({'Y':1, 'N':0})
    df['category_1'] = df['category_1'].map({'Y':1, 'N':0}) 
    df['yyyymm'] = df['purchase_date'].map(lambda x: str(x)[:7])
#     df['yyyymmdd'] = df['purchase_date'].map(lambda x: str(x)[:10])

In [5]:
# 未来の残高を計算する
auth1 = df_hist[df_hist.authorized_flag==1]
df_trans = pd.concat([df_new, auth1], axis=0)
use_cols = ['card_id', 'purchase_amount', 'installments', 'category_1', 'category_2', 'merchant_id', 'merchant_category_id', 'state_id', 'subsector_id', 'yyyymm']
df_trans = df_trans[use_cols]
df_trans = df_trans[df_trans['installments']>=2]
df_trans['installments'].value_counts()

2      643854
3      508821
4      162008
6      113808
5      103234
10      92318
12      38845
8       15732
7        8423
9        4309
11        609
999         8
Name: installments, dtype: int64

In [6]:
from time_utils import date_add_days

ym = 'yyyymm'
df_trans['yyyymm'] = pd.to_datetime(df_trans['yyyymm'])

ins_list = list(df_trans['installments'].value_counts().index)
df_list = []

for ins in ins_list:

    tmp = df_trans[df_trans['installments']==ins]
    
    # installments==2とした当月
    id_ym = tmp[[key, 'yyyymm', 'purchase_amount']].groupby([key, 'yyyymm'])['purchase_amount'].sum().reset_index()
    # 分割あたりのamount
    id_ym['purchase_amount'] = id_ym['purchase_amount'] / ins
    
    # 未来の月に対するamountをもったデータ
    df_ins_ym = id_ym[[key, 'yyyymm', 'purchase_amount']].copy()
    
    for i in range(int(ins-1)):
        df_ins_ym['yyyymm'] = df_ins_ym['yyyymm'].map(lambda x: date_add_days(x, 32) if int(str(x)[8:10])<=15 else date_add_days(x, 20))
        df_ins_ym['yyyymm'] = df_ins_ym['yyyymm'].map(lambda x: str(x)[:7])
        df_ins_ym['yyyymm'] = pd.to_datetime(df_ins_ym['yyyymm'])
        df_list.append(df_ins_ym.copy())
        
future_amt = pd.concat(df_list, axis=0, ignore_index=True)
print(future_amt.shape)
display(future_amt.head())

(3639662, 3)


,card_id,yyyymm,purchase_amount
0,C_ID_00007093c1,2017-03-01,0.018588
1,C_ID_00007093c1,2017-07-01,-0.523013
2,C_ID_00007093c1,2017-08-01,-0.260004
3,C_ID_00007093c1,2017-09-01,-0.324618
4,C_ID_00007093c1,2017-12-01,-0.196141


In [10]:
#========================================================================
# train_testにつけたhist_max_month, new_min_monthをつける
join_cols = ['hist_purchase_month_max', 'new_purchase_month_min', 'first_active_month']

print(future_amt.shape)
df_ins = future_amt.set_index(key).join(train_test.set_index(key)[join_cols])
print(df_ins.shape)
#========================================================================

#========================================================================
# new_month_minとの相対月をつけ、それを元にデータを絞り計算する
df_ins['yyyymm'] = pd.to_datetime(df_ins['yyyymm'])
df_ins['new_purchase_month_min'] = pd.to_datetime(df_ins['new_purchase_month_min'])
df_ins['new_month_lag']= (df_ins['new_purchase_month_min'].map(lambda x: x.year) - df_ins['yyyymm'].map(lambda x: x.year)) * 12 + (df_ins['new_purchase_month_min'].map(lambda x: x.month) - df_ins['yyyymm'].map(lambda x: x.month))
df_ins['new_month_lag'] *= -1
df_ins['hist_purchase_month_max'] = pd.to_datetime(df_ins['hist_purchase_month_max'])
df_ins['hist_month_lag']= (df_ins['hist_purchase_month_max'].map(lambda x: x.year) - df_ins['yyyymm'].map(lambda x: x.year)) * 12 + (df_ins['hist_purchase_month_max'].map(lambda x: x.month) - df_ins['yyyymm'].map(lambda x: x.month))
df_ins['hist_month_lag'] *= -1

# new_minとhist_maxにズレがあるとcard_idによって条件が異なってしまうので、newのデータがないcard_id以外はnewのlagにする
df_ins['new_month_lag'].fillna(-999, inplace=True)
df_ins['org_month_lag'] = df_ins[['new_month_lag', 'hist_month_lag']].min(axis=1)
df_ins['new_month_lag'] = df_ins['new_month_lag'].replace(-999, np.nan)
#========================================================================

(3639662, 3)
(3639662, 5)


In [12]:
# 期間別のデータを作る
df_ins_new = df_ins[df_ins['org_month_lag']>=0]
df_ins_past3 = df_ins[df_ins['org_month_lag']<0]
df_ins_past3 = df_ins_past3[df_ins_past3['org_month_lag']>=-3]
df_ins_past6 = df_ins[df_ins['org_month_lag']<0]
df_ins_past6 = df_ins_past6[df_ins_past6['org_month_lag']>=-6]

In [14]:
prefix = '215_ins'
prefix = '315_ins'
prefix = '415_ins'
df_list = [df_ins, df_ins_new, df_ins_past3, df_ins_past6]
fname_list = ['rem_ins', 'rem_ins_new', 'rem_ins_past3', 'rem_ins_past6']
agg_cols = [key, 'org_month_lag', 'cnt', 'purchase_amount']

def get_new_columns(name,aggs):
    return [name + '_' + k + '_' + agg for k in aggs.keys() for agg in aggs[k]]

for df, fname in zip(tqdm(df_list), fname_list):
    
    # Base
    train_test = utils.read_df_pkl('../input/base_term*0*')
    
    # FE
    df.reset_index(inplace=True)
    if key not in df.columns:
        df.rename(columns={'index':key}, inplace=True)
    df['cnt'] = 1
    df = df[agg_cols]
    # 月あたりの分割支払決済数
    tmp = df.groupby([key, 'org_month_lag']).sum().reset_index()
    aggs = {}
    aggs['cnt'] = ['sum', 'mean', 'max', 'min', 'std']
    aggs['purchase_amount'] = ['sum', 'mean', 'max', 'min', 'std']
    # 分割の最終月、初月、平均
    aggs['org_month_lag'] = ['max', 'min', 'mean']
    df_agg = tmp.groupby(key)['cnt'].agg(aggs)
    
    new_cols = get_new_columns(fname, aggs)
    df_agg.columns = new_cols
    
    train_test = train_test.join(df_agg)
    feat_cols = [col for col in train_test.columns if col.count('rem_ins')]

    elo_save_feature(df_feat=train_test[feat_cols], prefix=prefix)

100%|██████████| 4/4 [00:21<00:00,  5.27s/it]


In [16]:
prefix = '215_ins'
prefix = '315_ins'
prefix = '415_ins'
import glob
# #========================================================================
# # これは作成済のFeatureを読み込んで計算する
# #========================================================================
fname_list = ['rem_ins', 'rem_ins_new', 'rem_ins_past3', 'rem_ins_past6']

combi_list = [ 
    ['rem_ins_new', 'rem_ins_past3']
    ,['rem_ins_new', 'rem_ins_past6']
    ,['rem_ins_past3', 'rem_ins_past6']
]

comp_cols = [
     'cnt_sum'
    ,'cnt_max'
    ,'purchase_amount_sum'
    ,'purchase_amount_max'
    ,'org_month_lag_min'
    ,'org_month_lag_max'
]

tmp_feature_list = glob.glob(f'../features/1_first_valid/{prefix}*.gz')
                   
feature_list = []
for f in tmp_feature_list:
    if not(f.count(f'{prefix}')):continue
    for col in comp_cols:
        if f.count(col):
            feature_list.append(f)
            
base = utils.read_df_pkl('../input/base_first*0*')
p_list = utils.parallel_load_data(path_list=feature_list)
df_feat = pd.concat(p_list, axis=1)
train_test = pd.concat([base[key], df_feat], axis=1)

for (fm1, fm2) in tqdm(combi_list):
    for col in comp_cols:
#         203_pst_auth0_lag02_monthly_avg_purchase_amount_min
        try:
            train_test[f"ratio_{fm1}_{fm2}_{col}"] = train_test[f"{prefix}_{fm1}_{col}@"] / train_test[f"{prefix}_{fm2}_{col}@"]
            train_test[f"diff_{fm1}_{fm2}_{col}"] = train_test[f"{prefix}_{fm1}_{col}@"] - train_test[f"{prefix}_{fm2}_{col}@"]
        except KeyError:
            print(fm1, fm2, col)
            continue
print(train_test.shape)

ratio_diff_cols = [col for col in train_test.columns if col[:5]=='ratio' or col[:4]=='diff']
elo_save_feature(prefix, train_test[ratio_diff_cols], feat_check=False)

100%|██████████| 3/3 [00:00<00:00, 64.94it/s]


(325540, 61)
